In [1]:
from ctransformers import AutoModelForCausalLM

In [2]:
B_INST, E_INST = "[INST]", "[/INST]"
B_SYS, E_SYS = "<<SYS>>\n", "\n<</SYS>>\n\n"
BOS, EOS = "<s>", "</s>"

# fmt: off
DEFAULT_SYSTEM_PROMPT = """You are a helpful, respectful and honest assistant. Always answer as helpfully as possible, while being safe. Your \
answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. Please ensure\
 that your responses are socially unbiased and positive in nature.

If a question does not make any sense, or is not factually coherent, explain why instead of answering something not \
correct. If you don't know the answer to a question, please don't share false information."""
# fmt: on

In [3]:
from typing import Literal, TypedDict, List

Role = Literal["system", "user", "assistant"]

class Message(TypedDict):
    role: Role
    content: str
    
Dialog = List[Message]

In [4]:
dialog: Dialog = [
    {"role": "system","content": DEFAULT_SYSTEM_PROMPT,},
    {"role": "user", "content": "What is love?"},
]
if dialog[0]["role"] == "system":
    dialog = [
        {
            "role": dialog[1]["role"],
            "content": B_SYS + dialog[0]["content"] + E_SYS + dialog[1]["content"],
        }
    ] + dialog[2:]


In [5]:
print('loading model')
model = AutoModelForCausalLM.from_pretrained("../Bloke/model.gguf", model_type="llama", context_length=4096, threads=20)
print("Model loaded.")

loading model
Model loaded.


In [10]:
def get_tokens(dialog: Dialog) -> str:
    dialog_tokens = sum(
        [
            [f"{BOS}{B_INST} {(prompt['content']).strip()} {E_INST} {(answer['content']).strip()}{EOS}"]
            for prompt, answer in zip(dialog[::2], dialog[1::2],)
        ],
        [],
    )
    dialog_tokens += f"{BOS}{B_INST} {(dialog[-1]['content']).strip()} {E_INST}{EOS}",
    return ''.join(dialog_tokens)

In [11]:
get_tokens(dialog)

"<s>[INST] <<SYS>>\nYou are a helpful, respectful and honest assistant. Always answer as helpfully as possible, while being safe. Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. Please ensure that your responses are socially unbiased and positive in nature.\n\nIf a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information.\n<</SYS>>\n\nWhat is love? [/INST] Hello! I'm here to help answer your questions in a helpful, respectful, and honest manner. Your question about love is a complex and multifaceted one, and there are many different perspectives and interpretations on what love is.\n\nLove can be defined as a strong feeling of affection, care, and connection towards someone or something. It can manifest in many different ways, such as romantic love, familial love, platonic love, or s

In [14]:
out = ''
for token in model(get_tokens(dialog), stream=True, top_p=0.9, temperature=1.2, batch_size=1, max_new_tokens=512,):
    out += token
    print(token, end="", flush=True)
dialog.append({"role": "assistant", "content": out})
out = ''

 Certainly! Here's a small example of how love can provide a sense of belonging and acceptance:
Imagine that you are part of a diverse group of friends who all share a love for playing board games together. Each week, you gather at one of your homes to play games, share food, and just hang out. As you spend more time with this group, you begin to feel like you belong - like you're a valued and important part of the group.
One day, one of your friends shares that they have been struggling with depression and anxiety, and they haven't felt like themselves lately. The group rallies around them, offering words of support, listening to their concerns, and checking in on them regularly. You feel a sense of love and connection to this friend, knowing that they are not alone and that they have people who care about them.
In this example, the love and connection you feel with your friends provides a sense of belonging and acceptance. You know that you are valued and appreciated for who you are,

In [13]:
i = input().strip()
dialog.append({"role": "user", "content": i})